# TRAINING DATA CALCULATIONS

## IMPORTING

In [1]:
import math as mt
from math import cos
from math import sin
from math import radians
from math import degrees
import numpy as np
from numpy import array
from numpy import shape
import warnings
warnings.filterwarnings("ignore")

## CALCULATED Y BUS:

In [2]:
Y11 = 18.12
THETA11 = radians(-75.9)

Y22 = 15.58
THETA22 = radians(-74.3)

Y33 = 21.59
THETA33 = radians(-74.7)

Y12 = 6.06
THETA12 = radians(104)

Y21 = 6.06
THETA21 = radians(104)

Y13 = 12.13
THETA13 = radians(104)

Y31 = 12.13
THETA31 = radians(104)

Y23 = 9.57
THETA23 = radians(106.7)

Y32 = 9.57
THETA32 = radians(106.7)

## GIVEN DATA IN EACH BUS:

In [3]:
S = 13
G = 40

if (S<3 and S>25):
    P_wr = 0
elif (S>3 and S<16):
    P_wr= 0.3*((S-3)/13)
else:
    P_wr = 0.3
    
if (G<120):
    P_sr = 0.2*((G*G)/96000)
else:
    P_sr = 0.2*(G/800)
    
print("Solar generation is:", P_sr)
print("Wind generation is:", P_wr)

Solar generation is: 0.0033333333333333335
Wind generation is: 0.23076923076923078


In [4]:
# BUS 1 --> SLACK BUS

V1 = 1.04
DELTA1 = 0

# BUS 2 --> LOAD BUS

PL_2 = 0.5
P2_SP = P_wr - PL_2
Q2_SP = -0.3

# BUS 3 --> LOAD BUS

PL_3 = 0.35
P3_SP = P_sr - PL_3
Q3_SP = -0.4

## RECURSIVE FUNCTION TO GET THE VALUE OF V2, V3, DELTA2 AND DELTA3

In [5]:
# ASSUMING A INITIAL VALUE

V2 = 1
V3 = 1
DELTA2 = 0
DELTA3 = 0
i=1

# DEFINING THE FUNCTION

def cal_v():
    
    global V2, V3, DELTA2, DELTA3, i
    i = i+1
    
    # CALCULATING THE VALUES OF P2_CAL, P3_CAL, Q2_CAL AND Q3_CAL
    
    P2_CAL = ((V2*Y21*V1*cos(DELTA1-DELTA2+THETA21))+(V2*V2*Y22*cos(THETA22))+
                 (V2*Y23*V3*cos(DELTA3-DELTA2+THETA23)))
    P3_CAL = ((V3*Y31*V1*cos(DELTA1-DELTA3+THETA31))+(V3*V2*Y32*cos(DELTA2-DELTA3+THETA32))+
                 (V3*Y33*V3*cos(THETA33)))
    Q2_CAL = (-(V2*Y21*V1*sin(DELTA1-DELTA2+THETA21))-(V2*V2*Y22*sin(THETA22))-
                 (V2*Y23*V3*sin(DELTA3-DELTA2+THETA23)))
    Q3_CAL = (-(V3*Y31*V1*sin(DELTA1-DELTA3+THETA31))-(V3*V2*Y32*sin(DELTA2-DELTA3+THETA32))-
                 (V3*Y33*V3*sin(THETA33)))
    
    # CALCULATING THE VALUES OF DEL_P2, DEL_P3, DEL_Q2 AND DEL_Q3
    
    DEL_P2 = P2_SP - P2_CAL
    DEL_P3 = P3_SP - P3_CAL
    DEL_Q2 = Q2_SP - Q2_CAL
    DEL_Q3 = Q3_SP - Q3_CAL
    
    if (abs(DEL_P2)>0.01 and abs(DEL_Q2)>0.01 and abs(DEL_P3)>0.01 and abs(DEL_Q3)>0.01):
        
        M11 = ((V2*Y21*V1*sin(DELTA1-DELTA2+THETA21))+(V2*Y23*V3*sin(DELTA3-DELTA2+THETA23)))
        M12 = -(V2*Y23*V3*sin(DELTA3-DELTA2+THETA23))
        M13 = ((Y21*V1*cos(DELTA1-DELTA2+THETA21))+(2*V2*Y22*cos(THETA22)+Y23*V3*cos(DELTA3-DELTA2+THETA23)))
        M14 = (V2*Y23*cos(DELTA3-DELTA2+THETA23))
        M31 = ((V2*Y21*V1*cos(DELTA1-DELTA2+THETA21))+(V2*Y23*V3*cos(DELTA3-DELTA2+THETA23)))
        M32 = -(V2*Y23*V3*cos(DELTA3-DELTA2+THETA23))
        M33 = ((-Y21*V1*sin(DELTA1-DELTA2+THETA21))-(2*V2*Y22*sin(THETA22))-(Y23*V3*sin(DELTA3-DELTA2+THETA23)))
        M34 = (-V2*Y23*sin(DELTA3-DELTA2+THETA23))
        M21 = (-V3*Y32*V2*sin(DELTA2-DELTA3+THETA32))
        M22 = ((V3*Y31*V1*sin(DELTA1-DELTA3+THETA31))+(V3*Y32*V2*sin(DELTA2-DELTA3+THETA32)))
        M23 = (V3*Y32*cos(DELTA2-DELTA3+THETA32))
        M24 = ((Y31*V1*cos(DELTA1-DELTA3+THETA31))+(Y32*V2*cos(DELTA2-DELTA3+THETA32))+(2*V3*Y33*cos(THETA33)))
        M41 = -(V3*Y32*V2*cos(DELTA2-DELTA3+THETA32))
        M42 = ((V3*Y31*V1*cos(DELTA1-DELTA3+THETA31))+(V3*Y32*V2*cos(DELTA2-DELTA3+THETA32)))
        M43 = -(V3*Y32*sin(DELTA2-DELTA3+THETA32))
        M44 = (-(Y31*V1*sin(DELTA1-DELTA3+THETA31))-(Y32*V2*sin(DELTA2-DELTA3+THETA32))-(2*V3*Y33*sin(THETA33)))
        
        J = array([[M11, M12, M13, M14], [M21, M22, M23, M24], [M31, M32, M33, M34], [M41, M42, M43, M44]])
        #print("\nThe Jacobines matrix is:\n\n",J,"\n")
        J_INV = np.linalg.inv(J)
        #print("The inverse of Jacobines matrix is:\n\n",J_INV,"\n\n->>>The shape of matrix is",shape(J_INV),"\n")
        
        M_DEL = array([[DEL_P2],[DEL_P3],[DEL_Q2],[DEL_Q3]])
        #print("The matrix of del is:\n\n",M_DEL,"\n\n->>>The shape of the matrix is",shape(M_DEL),"\n")
        
        M = np.dot(J_INV,M_DEL)
        #print("The final matrix is:\n\n",M,"\n")
        
        V2 = V2+ M[2,0]
        V3 = V3+ M[3,0]
        DELTA2 = DELTA2+ M[0,0]
        DELTA3 = DELTA3+ M[1,0]
        
        # Calling the Function
        
        cal_v()
        
    else:
            print("\nV2 = ",V2,"\nV3 = ",V3,"\nDELTA2 = ",degrees(DELTA2),"\nDELTA3 = ",degrees(DELTA3))
            print("\nThe number of iterations:",i)

## CALLING THE RECURSIVE FUNCTION

In [6]:
cal_v()


V2 =  0.9986771594797009 
V3 =  1.0050838952744343 
DELTA2 =  -1.5550556286522268 
DELTA3 =  -1.3987558443073251

The number of iterations: 3
